In [2]:
import IJulia

In [4]:
import Pkg
Pkg.add("Primes")
Pkg.add("Statistics")
Pkg.add("Distributions")
Pkg.add("StatsFuns")

  Updating registry at `C:\Users\noname\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0%0.0 %.0 %5 %1 %2 %22.3 %>                          ]  34.6 % %                  ]  53.6 %Fetching: [===========================>             ]  65.2 %         ]  77.3 %]  87.4 %99.7 % % Resolving package versions...
 Installed Missings ─────────────── v0.4.3
 Installed Compat ───────────────── v2.2.0
 Installed ArrayInterface ───────── v2.0.0
 Installed Primes ───────────────── v0.4.0
 Installed StaticArrays ─────────── v0.12.1
 Installed Highlights ───────────── v0.4.5
 Installed Optim ────────────────── v0.19.5
 Installed GR ───────────────────── v0.44.0
 Installed MacroTools ───────────── v0.5.2
 Installed OffsetArrays ─────────── v0.11.2
 Installed FFTW ─────────────────── v1.1.0
 Installed Mustache ─────────────── v1.0.0
 Installed Calculus ─────────────── v0.5.1
 Installed NLSolversBase

In [6]:
Pkg.add("Distances")

 Resolving package versions...
  Updating `C:\Users\noname\.julia\environments\v1.2\Project.toml`
  [b4f34e82] + Distances v0.8.2
  Updating `C:\Users\noname\.julia\environments\v1.2\Manifest.toml`
 [no changes]


## Correlation matrix

### Unit square-grid-array, indexed by Morton order

Assume the level of Morton indexing is 1. Fix m = 64, and n = 256, 1024, 4096, 16384

In [47]:
using Distances

n = [256, 1024, 4096, 16384]
c = [0.3, 0.1, 0.03]
function Mor(n::Int64, c::Float64) 
    a=trunc(Int,sqrt(n))-1
    A=Vector{Float64}(undef,a+1)   ## grid : x-coord
    B=Vector{Float64}(undef,a+1)   ## grid : y-coord
    C=zeros(a+1,a+1)   ## Index by Morton order
    D=Matrix{Float64}(undef,n,n)
    for i=1:(a+1)
        A[i] = (i-1)/a 
        B[i] = 1-(i-1)/a
        for j=1:(a+1)
            C[i,j] = trunc(Int,(a+1)*(i-1)+j)
        end
    end

    for i=1:n
        for j=i:n
        D[i,j] = exp(-euclidean([A[findall(x->x==i,C)[1][1]], B[findall(x->x==i, C)[1][2]]],[A[findall(x->x==j,C)[1][1]], B[findall(x->x==j, C)[1][2]]])/c)
        D[j,i] = D[i,j]
        end
    end
    return D
end


Mor (generic function with 1 method)

## Block Reordering

Fix $m = 64$, $a = -\inf$, $b \sim$ $U(0,1)$, $d = 1$(while simulation)

In [55]:
function Blockreorder(n::Int64, c::Float64, ind::Array{Float64,1}, a::Array{Float64,1}, b::Array{Float64,1}, m::Int64)
    P=Array{Float64}(undef, trunc(Int, n/m))
    for j=1:m:(n-m+1)
        subind=ind[j:(j+m-1)]
        A = Mor(n,c)[subind, subind]
        a0 = a[subind]
        b0 = b[subind]
        P = [P,RCMVN(A, a0, b0, 1).P]   ## Required RCMVN
    end
end


Blockreorder (generic function with 1 method)